In [1]:
# !pip install spade==4.0.0rc4
# !pip install click

# Gossiping con protocolo push donde se converge al valor máximo

El envio de mensajes es solo a un vecino aleatorio

**Recuerda cambiar el nombre de los agentes**

In [2]:
import datetime
import json
import random
import time
import click
import asyncio
import spade


class PushAgent(spade.agent.Agent):

    async def setup(self):
        self.value = random.randint(1, 1000)

        #await asyncio.sleep(15)

        start_at = datetime.datetime.now() + datetime.timedelta(seconds=5)
        self.add_behaviour(self.PushBehaviour(period=2, start_at=start_at))
        template = spade.template.Template(metadata={"performative": "PUSH"})
        self.add_behaviour(self.RecvBehaviour(), template)

        #print("{} ready.".format(self.name))

    def add_value(self, value):
        # seleccion del valor adecuado entre el propio y el nuevo
        #print(self.value, value)
        self.value = max(self.value, value)

    def add_contacts(self, contact_list):
        self.contacts = [c.jid for c in contact_list if c.jid != self.jid]
        self.length = len(self.contacts)

    # comportamiento encargado de enviar el mensaje push
    class PushBehaviour(spade.behaviour.PeriodicBehaviour):

        async def run(self):
            # el numero de amigos está fijado a 1, se puede modificar
            k=1
            #print("{} period with k={}!".format(self.agent.name, k))
            random_contacts = random.sample(self.agent.contacts, k)
            #print("{} sending to {}".format(self.agent.name, [x.localpart for x in random_contacts]))
            
            # se envia el mensaje con el dato a los k amigos seleccionados
            for jid in random_contacts:
                body = json.dumps({"value": self.agent.value, "timestamp": time.time()})
                msg = spade.message.Message(to=str(jid), body=body, metadata={"performative": "PUSH"})
                await self.send(msg)

    # comportamiento encargado de gestionar la llegada de un mensaje push
    class RecvBehaviour(spade.behaviour.CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=2)
            if msg:
                body = json.loads(msg.body)
                # llamamos al método encargado de decidir si actualiza el dato o no
                self.agent.add_value(body["value"])

                #print("[{}] <{}>".format(self.agent.name, self.agent.value))






Using slower stringprep, consider compiling the faster cython/libidn one.


In [3]:


count=50
agents = []
print("Creating {} agents...".format(count))
for x in range(1, count + 1):
    print("Creating agent {}...".format(x))
    # nos guardamos la lista de agentes para poder visualizar el estado del proceso gossiping
    # el servidor está fijado a gtirouter.dsic.upv.es, si se tiene un serviodor XMPP en local, se puede sustituir por localhost
    agents.append(PushAgent("push_agent_1626_{}@localhost".format(x), "test"))

# este tiempo trata de esperar que todos los agentes estan registrados, depende de la cantidad de agentes que se lancen
await asyncio.sleep(3)

# se le pasa a cada agente la lista de contactos
for ag in agents:
    ag.add_contacts(agents)
    ag.value = 0

# se lanzan todos los agentes
agentes = []
for ag in agents:
    agentes.append(ag.start())
    
await asyncio.gather(*agentes)

# este tiempo trata de esperar que todos los agentes estan ready, depende de la cantidad de agentes que se lancen


# este bucle imprime los valores que almacena cada agente y termina cuando todos tienen el mismo valor (consenso)
while True:
    try:
        await asyncio.sleep(1)
        status = [ag.value for ag in agents]
        print("STATUS: {}".format(status))
        if len(set(status)) <= 1:
            print("Gossip done.")
            break
    except KeyboardInterrupt:
        break

# se para a todos los agentes
for ag in agents:
    await ag.stop()
print("Agents finished")

Creating 50 agents...
Creating agent 1...
Creating agent 2...
Creating agent 3...
Creating agent 4...
Creating agent 5...
Creating agent 6...
Creating agent 7...
Creating agent 8...
Creating agent 9...
Creating agent 10...
Creating agent 11...
Creating agent 12...
Creating agent 13...
Creating agent 14...
Creating agent 15...
Creating agent 16...
Creating agent 17...
Creating agent 18...
Creating agent 19...
Creating agent 20...
Creating agent 21...
Creating agent 22...
Creating agent 23...
Creating agent 24...
Creating agent 25...
Creating agent 26...
Creating agent 27...
Creating agent 28...
Creating agent 29...
Creating agent 30...
Creating agent 31...
Creating agent 32...
Creating agent 33...
Creating agent 34...
Creating agent 35...
Creating agent 36...
Creating agent 37...
Creating agent 38...
Creating agent 39...
Creating agent 40...
Creating agent 41...
Creating agent 42...
Creating agent 43...
Creating agent 44...
Creating agent 45...
Creating agent 46...
Creating agent 47...


DisconnectedException: Error during the connection with the server